# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [38]:
import pandas as pd
import time

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

filename_features = "../_Dados/UCI HAR Dataset/features.txt"
filename_labels = "../_Dados/UCI HAR Dataset/activity_labels.txt"

filename_subtrain = "../_Dados/UCI HAR Dataset/train/subject_train.txt"
filename_xtrain = "../_Dados/UCI HAR Dataset/train/X_train.txt"
filename_ytrain = "../_Dados/UCI HAR Dataset/train/y_train.txt"

filename_subtest = "../_Dados/UCI HAR Dataset/test/subject_test.txt"
ffilename_xtest = "../_Dados/UCI HAR Dataset/test/X_test.txt"
filename_ytest = "../_Dados/UCI HAR Dataset/test/y_test.txt"

features = pd.read_csv(filename_features, header=None, names=['nome_var'], squeeze=True, sep="#")
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'], squeeze=True)
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features.tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'], squeeze=True)
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=features.tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

C:\Users\rodpr\AppData\Local\Temp\ipykernel_29148\3368985066.py:22: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  features = pd.read_csv(filename_features, header=None, names=['nome_var'], squeeze=True, sep="#")
C:\Users\rodpr\AppData\Local\Temp\ipykernel_29148\3368985066.py:25: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'], squeeze=True)
C:\Users\rodpr\AppData\Local\Temp\ipykernel_29148\3368985066.py:29: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'], squeeze=True)


## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [6]:
%%time
ccp_alpha=0.001
clf = DecisionTreeClassifier(random_state=2360873, ccp_alpha=ccp_alpha).fit(X_train, y_train)

train_score = clf.score(X_train, y_train)
test_score = clf.score(X_test, y_test)

print(f'Acurácia da árvore na base de treino:    {train_score *100:.1f}')
print(f'Acurácia da árvore na base de teste:     {test_score*100:.1f}')

Acurácia da árvore na base de treino:    97.6
Acurácia da árvore na base de teste:     87.9
CPU times: total: 3.19 s
Wall time: 4.46 s


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [24]:
%%time
# Treinando o PCA (Principal Component Analysis) com 1 componente (n_components=1) usando os dados de treinamento (X_train).
prcomp = PCA(n_components=1).fit(X_train) # treinando a árvore com a principal componente
# Transformando os dados de treinamento (X_train) usando o PCA treinado (rcomp) para obter as principais componentes.
pc_train = prcomp.transform(X_train)
# Transformando os dados de teste (X_test) usando o PCA treinado (rcomp) para obter as principais componentes.
pc_test  = prcomp.transform(X_test)

#Criando um DataFrame para armazenar as principal componente dos dados de treinamento (pc_train)
pc_train = pd.DataFrame(pc_train, columns = ['cp1'])
#Criando um DataFrame para armazenar as principal componente dos dados de teste(pc_train)
pc_test  = pd.DataFrame( pc_test, columns = ['cp1'])


CPU times: total: 422 ms
Wall time: 184 ms


In [25]:
pc_train.head()

,cp1
0,-5.520280
1,-5.535350
2,-5.474988
3,-5.677232
4,-5.748749


In [26]:
pc_test.head()

,cp1
0,-2.686743
1,-4.331255
2,-4.985360
3,-5.099876
4,-5.023000


In [28]:
%%time
#treinando a árvore com a componente principal como variável explicativa
clf = DecisionTreeClassifier(random_state=1234).fit(pc_train, y_train)

train_score = clf.score(pc_train, y_train)
test_score = clf.score(pc_test, y_test)

print(f'Acurácia da árvore na base de treino:    {train_score *100:.1f}')
print(f'Acurácia da árvore na base de teste:     {test_score*100:.1f}')

Acurácia da árvore na base de treino:    100.0
Acurácia da árvore na base de teste:     41.0
CPU times: total: 15.6 ms
Wall time: 28.7 ms


Com uma só componente principal o modelo ficou overfittado, conforme  observado pelas acurácias da árvore na base de treino e na base de teste.

## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [46]:
%%time
num_components = [1, 2, 5, 10, 50, 80, 100, 200, 300, 400, 450, 561]

for num in num_components:
    start_time = time.time()  # Armazena o tempo de início do loop
    
    prcomp = PCA(n_components=num).fit(X_train) # treinando a árvore com a principal componente
    # Transformando os dados de treinamento (X_train) usando o PCA treinado (rcomp) para obter as principais componentes.
    pc_train = prcomp.transform(X_train)
    # Transformando os dados de teste (X_test) usando o PCA treinado (rcomp) para obter as principais componentes.
    pc_test  = prcomp.transform(X_test)
    
    #treinando a árvore com a componente principal como variável explicativa
    clf = DecisionTreeClassifier(random_state=1234).fit(pc_train, y_train)

    train_score = clf.score(pc_train, y_train)
    test_score = clf.score(pc_test, y_test)
    
    end_time = time.time()  # Armazena o tempo de término do loop
    elapsed_time = end_time - start_time  # Calcula o tempo total do loop
    
    print(f'Árvore treinada com {num} componentes principais:\n')
    print(f'Acurácia da árvore na base de treino:    {train_score *100:.1f}')
    print(f'Acurácia da árvore na base de teste:     {test_score*100:.1f}')
    print(f'Tempo levado: {elapsed_time:.2f} segundos')
    print('-----------------------------------------------')




Árvore treinada com 1 componentes principais:

Acurácia da árvore na base de treino:    100.0
Acurácia da árvore na base de teste:     41.0
Tempo levado: 0.21 segundos
-----------------------------------------------
Árvore treinada com 2 componentes principais:

Acurácia da árvore na base de treino:    100.0
Acurácia da árvore na base de teste:     51.7
Tempo levado: 0.21 segundos
-----------------------------------------------
Árvore treinada com 5 componentes principais:

Acurácia da árvore na base de treino:    100.0
Acurácia da árvore na base de teste:     75.3
Tempo levado: 0.24 segundos
-----------------------------------------------
Árvore treinada com 10 componentes principais:

Acurácia da árvore na base de treino:    100.0
Acurácia da árvore na base de teste:     79.0
Tempo levado: 0.33 segundos
-----------------------------------------------
Árvore treinada com 50 componentes principais:

Acurácia da árvore na base de treino:    100.0
Acurácia da árvore na base de teste:    

## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

A acurácia na base de teste  vai aumentando bastante conforme incrementamos mais componentes principais, contudo, a partir de 50 componentes principais, adicionar mais componentes não tem mais efeito na melhoria da acurácia da base de teste (acurácia fica oscilando);

O tempo de execução vai aumentando conforme aumenta o número de componentes principais consideradas.
